<h5> Ekrem Çetinkaya S004228 </h5>
<hr />
<h1> <strong> Cifar-10 Evaluator </strong> </h1>
<hr />

<h5> Imports </h5>

In [1]:
import tensorflow as tf
import numpy as np
import mcNemar_test
import pickle
import read_data

Model 1 Right, Model 2 Wrong : 1709 
Model 1 Wrong, Model 2 Right : 1069 
Model 1 Wrong, Model 2 Wrong : 2066 
Model 1 Right, Model 2 Right : 6225 


<h5> Count number of hidden parameters </h5>

In [2]:
def display_number_of_parameters():
    total_parameters = 0
    for variable in tf.trainable_variables():  
    	local_parameters=1
    	shape = variable.get_shape()  #getting shape of a variable
    	for i in shape:
        	local_parameters*=i.value  #mutiplying dimension values
    	total_parameters+=local_parameters
    print("Total number of hidden parameters: %i" % total_parameters)

<h5> Choose network to test </h5>

In [3]:
#model_path = ".\\cifar_model_one"
model_path = ".\\cifar_model_two"

<h5> Test Function </h5>

In [4]:
batch_size = 100

def evaluate_model():
    test_images, test_labels = pickle.load(open('preprocess_training.p', mode='rb'))
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(model_path + '.meta')
        loader.restore(sess, model_path)

        # Get tensors from loaded model
        loaded_images = loaded_graph.get_tensor_by_name('image:0')
        loaded_labels = loaded_graph.get_tensor_by_name('label:0')
        loaded_logits = loaded_graph.get_tensor_by_name('logits:0')

        display_number_of_parameters()
        true_count = 0
        prediction_results = np.zeros(10000)
        current_batch = 0
        for train_image_batch, train_label_batch in read_data.batch_images_labels(test_images, test_labels, batch_size):
            predictions = sess.run(tf.nn.top_k(tf.nn.softmax(loaded_logits), 1), feed_dict={loaded_images: train_image_batch,
                                                                              loaded_labels: train_label_batch})
            # This part will be used while conducting McNemar Test
            for i in range (0, batch_size - 1):
                prediction_label = int(predictions[1][i])
                correct_label = int(test_labels[current_batch + i].tolist().index(1))
                if(prediction_label == correct_label):
                    true_count += 1
                    prediction_results[i + current_batch] = 1
            current_batch += batch_size
        #Save prediction numpy array to use in mcnemar test
        mcNemar_test.save_mcnemar_array(model_path, prediction_results)
        accuracy = (true_count / 10000) * 100
        print('Testing Accuracy: {0:.2f} %\n'.format(accuracy))

<h3> Run the test </h3>

In [5]:
evaluate_model()

INFO:tensorflow:Restoring parameters from .\cifar_model_two


Total number of hidden parameters: 439434


Testing Accuracy: 62.25 %



<h3> McNemar Test </h3>
<hr />
<h5> DO NOT RUN THIS BEFORE EVALUATING BOTH MODELS </h5>
In order to run McNemar test, both models need to be evaluated before.

In [6]:
mcNemar_test.run_mcnemar_test()

Model 1 Right, Model 2 Wrong : 1709 
Model 1 Wrong, Model 2 Right : 1069 
Model 1 Wrong, Model 2 Wrong : 2066 
Model 1 Right, Model 2 Right : 6225 
